In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_json(open("./data/train.json", "r"))

In [7]:
any(pd.to_datetime(train["created"]).dt.year != 2016)

False

In [ ]:
pd.

In [ ]:
t = train.groupby('manager_id')['manager_id'].value_counts()

In [ ]:
s=train.groupby(['manager_id','interest_level']).size().unstack()

In [ ]:
import numpy as np

In [ ]:
from numba import vectorize, float32, int32
from numba.types import UniTuple

In [ ]:
@autojit
def intlevel_freqcount(a,b,c):
    a,b,c = np.nan_to_num([a,b,c])
    tot = sum([a,b,c])
    return pd.Series([a/tot,b/tot,c/tot,tot])
    

In [ ]:
intlevel_freqcount(1,2,5)

In [ ]:
s.apply(np.nan_to_num)

In [ ]:
s.reset_index().apply(np.nan_to_num).merge(t,how='left',left_on='manager_id',right_index=True)

In [ ]:
t

In [ ]:
    df_manager = (pd.concat([train['manager_id'], 
                           pd.get_dummies(train['interest_level'])], axis = 1)
                                        .groupby('manager_id')
                                        .mean()
                                        .rename(columns = lambda x: 'manager_percent_' + x)
                                           )


In [ ]:
df_manager

In [ ]:
t = train.groupby('manager_id')['manager_id'].value_counts()

In [ ]:


pd.concat([train['manager_id'],pd.get_dummies(train['interest_level'])], axis = 1).groupby('manager_id').mean()

In [ ]:
df_manager['test']=train.groupby('manager_id').size()

In [ ]:
df_manager

In [ ]:
df_mngr = (pd.concat([train['manager_id'], 
                           pd.get_dummies(train['interest_level'])], axis = 1)
                                        .groupby('manager_id')
                                        .transform('mean')
                                        .rename(columns = lambda x: 'mngr_percent_' + x)
                                           )



In [ ]:
df_mngr

In [ ]:
# get ixes for unranked managers...
unrkd_mngrs_ixes = df_mngr['manager_count']<20
# ... and ranked ones
rkd_mngrs_ixes = ~unrkd_mngrs_ixes

# compute mean values from ranked managers and assign them to unranked ones
mean_val = df_mngr.loc[rkd_mngrs_ixes,
                           ['mngr_percent_high',
                            'mngr_percent_low',
                            'mngr_percent_medium',
                            'manager_skill']].mean()
df_mngr.loc[unrkd_mngrs_ixes, ['mngr_percent_high',
                                    'mngr_percent_low',
                                    'mngr_percent_medium',
                                    'manager_skill']] = mean_val.values

In [ ]:
train['description'][100004]

In [31]:
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
from sklearn import metrics
from hdbscan import HDBSCAN
import time
%matplotlib inline

In [ ]:
# define the number of kilometers in one radian
kms_per_radian = 6371.0088

In [25]:
coords = train[['latitude','longitude']]

In [ ]:
coords

In [ ]:
# define epsilon as 200m, converted to radians for use by haversine
epsilon = 0.2 / kms_per_radian

In [ ]:
start_time = time.time()
db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine', n_jobs=-1).fit(np.radians(coords))
cluster_labels = db.labels_

# get the number of clusters
num_clusters = len(set(cluster_labels))

# all done, print the outcome
message = 'Clustered {:,} points down to {:,} clusters, for {:.1f}% compression in {:,.2f} seconds'
print(message.format(len(train), num_clusters, 100*(1 - float(num_clusters) / len(train)), time.time()-start_time))
#print('Silhouette coefficient: {:0.03f}'.format(metrics.silhouette_score(coords, cluster_labels)))

In [ ]:
clusters = pd.Series([coords[cluster_labels==n] for n in range(num_clusters)])
print(clusters)
centermost_points = clusters.map(_get_centermost_point)

In [ ]:
def _get_centermost_point(cluster):
        centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
        centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
        return tuple(centermost_point)
    
def _trans(df):
        
        coords = df[['latitude','longitude']].as_matrix()
        
        start_time = time.time()
        db = HDBSCAN(min_samples=1,
                    metric='haversine',
                    core_dist_n_jobs=-1,
                    memory='./__pycache__/' 
                   ).fit(coords)
        cluster_labels = db.labels_
        
        # get the number of clusters
        num_clusters = len(set(cluster_labels))
        message = 'Clustered {:,} points down to {:,} clusters, for {:.1f}% compression in {:,.2f} seconds'
        print(message.format(len(df), num_clusters, 100*(1 - float(num_clusters) / len(df)), time.time()-start_time))
        
        # turn the clusters in to a pandas series, where each element is a cluster of points
        clusters = pd.Series([coords[cluster_labels==n] for n in range(num_clusters)])
        print(clusters)
        centermost_points = clusters.map(_get_centermost_point)
        
        #Retrieve the lists of centermost points
        new_lat, new_long = zip(*centermost_points)
        return df.assign(
            cluster_lat = new_lat,
            cluster_long = new_long
        )

In [4]:
test = pd.read_json(open("./data/test.json", "r"))

In [ ]:
_trans(test)

In [1]:
from hdbscan import HDBSCAN

In [ ]:
        coords = train[['latitude','longitude']]
        
        start_time = time.time()
        db = HDBSCAN(min_samples=1,
                    metric='haversine',
                    core_dist_n_jobs=-1,
                    memory='./__pycache__/'
                   ).fit(coords)
        cluster_labels = db.labels_
        
        # get the number of clusters
        num_clusters = cluster_labels.max()
        message = 'Clustered {:,} points down to {:,} clusters, for {:.1f}% compression in {:,.2f} seconds'
        print(message.format(len(train), num_clusters, 100*(1 - float(num_clusters) / len(train)), time.time()-start_time))
        
        # turn the clusters in to a pandas series, where each element is a cluster of points
        clusters = pd.Series([coords.iloc[cluster_labels==n] for n in range(num_clusters)])

In [ ]:
cluster_labels

In [ ]:
cluster_labels.max()

In [ ]:
coords_new = coords

In [ ]:
c = coords.assign( cluster = db.labels_)

In [ ]:
gb = c.groupby('cluster')['latitude','longitude']

In [ ]:
cluster_centers = gb.agg(lambda x: _get_centermost_point(x.values))

In [29]:
def _get_centermost_point(cluster):
        centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
        centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
        lat,long,label = centermost_point
        return lat,long

In [ ]:
c.merge(cluster_centers, left_on='cluster', right_index=True,how='left',suffixes=('', '_cluster'))

In [ ]:
cluster_centers

In [20]:
coords_test = test[['latitude','longitude']]

In [ ]:
db.approximate_predict(coords_test)

In [11]:
        db = HDBSCAN(min_samples=1,
                    metric='haversine',
                    algorithm='boruvka_balltree',
                    core_dist_n_jobs=-1,
                    memory='./__pycache__/',
                    prediction_data=True
                   )

In [23]:
coords = train[['latitude','longitude']]

In [26]:
train2 = train.assign( cluster = db.fit_predict(coords) )

In [34]:
train2[['cluster','latitude','longitude']].groupby('cluster')['latitude','longitude'].agg(lambda x: _get_centermost_point(x.values))

,latitude,longitude
cluster,,
-1,-1.0,40.7464
0,0.0,0.0000
1,1.0,40.7128
2,2.0,40.7179
3,3.0,40.7211
4,4.0,40.7182
5,5.0,40.7183
6,6.0,40.7169
7,7.0,40.7166


In [9]:
from hdbscan import approximate_predict

In [21]:
a = approximate_predict(db,coords_test)

/usr/lib/python3.6/site-packages/hdbscan/prediction.py:187: RuntimeWarning: divide by zero encountered in double_scalars
  lambda_ = 1. / mr_distances[nn_index]
/usr/lib/python3.6/site-packages/hdbscan/prediction.py:319: RuntimeWarning: invalid value encountered in double_scalars
  prob = (lambda_ / max_lambda)


In [22]:
a[0].shape

(74659,)

In [17]:
a[1]==1

array([ True, False, False, ..., False, False, False], dtype=bool)